In [1]:
!gpustat


qgpu01                    Mon Mar  3 17:39:50 2025  525.105.17
[0] NVIDIA A100 80GB PCIe | 34°C,   0 % |     0 / 81920 MB |
[1] NVIDIA A100 80GB PCIe | 33°C,   0 % |     0 / 81920 MB |


In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import io
import json
import pickle
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
from PIL import Image
import natsort
import os

import dotenv
dotenv.load_dotenv("../.env")

import torch
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

import transformers
from transformers import AutoModel, AutoTokenizer
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from utils import *


/home/ga59how2/projects/ml24env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
bias_name_map = {
    "Age":      "age",
    "Disability_status":        "disability",
    "Gender_identity":      "gender",
    "Nationality":      "nationality",
    "Physical_appearance":      "appearance",
    "Race_ethnicity":       "race",
    "Religion":     "religion",
    "SES":      "ses",
    "Sexual_orientation":       "orientation",
    "Race_x_gender":        "race_gender",
    "Race_x_SES":       "race_ses",
}


In [4]:
target_groups = ["age", "appearance", "gender", "race", "religion", "ses"]
# target_group = target_groups[0]

# df_path = F"../bias_images/bias_{target_group}/{target_group}_without_core_context.csv"
# if not os.path.exists(df_path):
#     df_path = F"../bias_images/bias_{target_group}/{target_group}_with_core_context.csv"
# df_target_table = pd.read_csv(df_path)
# df_target_table.head(3)

In [5]:
all_df_scores = {}

for target_group in target_groups:
    pickle_path = F"../scores_pickles/df_scores_{target_group}.pkl"
    df_scores = pd.read_pickle(pickle_path)
    all_df_scores[target_group] = df_scores

print(all_df_scores.keys())
df_scores = pd.concat(all_df_scores.values(), ignore_index=True)
df_scores

# only keep the rows that df_scores["scores"][i][0][0] is larger than 0.7
df_scores_filtered = df_scores[df_scores["scores"].apply(lambda x: x[0][0] > 0.8)]
df_scores_filtered


dict_keys(['age', 'appearance', 'gender', 'race', 'religion', 'ses'])


category question_index                         label  \
0         age              1               The grandfather   
3         age              1               The grandmother   
5         age              2               The grandfather   
7         age              2               The grandmother   
8         age              3               The elderly man   
...       ...            ...                           ...   
2468      ses             23                    The lawyer   
2469      ses             23                   The chemist   
2470      ses             23               The farm worker   
2471      ses             23                   The cashier   
2472      ses             24  The one on public assistance   

                                                 scores  
0     [(0.86846, The grandfather_1.png), (0.8677, Th...  
3     [(0.92414, The grandmother_40.png), (0.92207, ...  
5     [(0.88775, The grandfather_46.png), (0.86975, ...  
7     [(0.92719, The grandmother_36.png), (0.90985, ...  
8     [(0.97165, The elderly man_40.png), (0.97104, ...  
...                                                 ...  
2468  [(0.95964, The lawyer_32.png), (0.9483, The la...  
2469  [(0.9751, The chemist_23.png), (0.97292, The c...  
2470  [(0.97512, The farm worker_46.png), (0.97512, ...  
2471  [(0.93716, The cashier_12.png), (0.92073, The ...  
2472  [(0.93824, The one on public assistance_6.png)...  

[1081 rows x 4 columns]

In [7]:
df_scores[df_scores["category"] == "age"]

category question_index                     label  \
0        age              1           The grandfather   
1        age              1         The granddaughter   
2        age              1              The grandson   
3        age              1           The grandmother   
4        age              2              The grandson   
..       ...            ...                       ...   
232      age             25       The middle-aged man   
233      age             25     The college sophomore   
234      age             25    The high school junior   
235      age             25       The college student   
236      age             25  The high school freshman   

                                                scores  
0    [(0.86846, The grandfather_1.png), (0.8677, Th...  
1    [(0.46208, The granddaughter_40.png), (0.45685...  
2    [(0.5504, The grandson_35.png), (0.48729, The ...  
3    [(0.92414, The grandmother_40.png), (0.92207, ...  
4    [(0.58208, The grandson_48.png), (0.49858, The...  
..                                                 ...  
232  [(0.93515, The middle-aged man_17.png), (0.926...  
233  [(0.67913, The college sophomore_34.png), (0.6...  
234  [(0.73563, The high school junior_24.png), (0....  
235  [(0.94659, The college student_45.png), (0.941...  
236  [(0.71477, The high school freshman_15.png), (...  

[237 rows x 4 columns]

In [8]:
all_top_images = []
for i, row in df_scores_filtered.iterrows():
    if row["category"] == "gender":
        continue

    img_path = Path(F"../bias_images/bias_{row['category']}/generated_images_{row['category']}/{row['category']}_{row['question_index']}")
    for score_img_pair in row["scores"]:
        if score_img_pair[0] > 0.8 and (img_path / score_img_pair[1]).exists():
            all_top_images.append(img_path / score_img_pair[1])

print(len(all_top_images), all_top_images[:5])

# sample 4000 images
np.random.seed(0)
np.random.shuffle(all_top_images)
all_images = all_top_images[:2000]

# make image pairs without the same name
all_pairs = []
for i in range(0, len(all_images), 2):
    if all_images[i].name.split("_")[0] != all_images[i+1].name.split("_")[0]:
        all_pairs.append((all_images[i], all_images[i+1]))

len(all_pairs), all_pairs[:3]

20923 [PosixPath('../bias_images/bias_age/generated_images_age/age_1/The grandfather_1.png'), PosixPath('../bias_images/bias_age/generated_images_age/age_1/The grandfather_0.png'), PosixPath('../bias_images/bias_age/generated_images_age/age_1/The grandfather_43.png'), PosixPath('../bias_images/bias_age/generated_images_age/age_1/The grandfather_14.png'), PosixPath('../bias_images/bias_age/generated_images_age/age_1/The grandfather_30.png')]


(983,
 [(PosixPath('../bias_images/bias_race/generated_images_race/race_12/The Asian woman_48.png'),
   PosixPath('../bias_images/bias_ses/generated_images_ses/ses_23/The sales clerk_13.png')),
  (PosixPath('../bias_images/bias_ses/generated_images_ses/ses_14/The pharmacist_16.png'),
   PosixPath('../bias_images/bias_appearance/generated_images_appearance/appearance_16/The one who is overweight_37.png')),
  (PosixPath('../bias_images/bias_race/generated_images_race/race_3/The Asian man_47.png'),
   PosixPath('../bias_images/bias_ses/generated_images_ses/ses_13/The professor_37.png'))])

In [9]:
# Open images
headline = Image.open("headline_ab.png")  # 512x64
headline = Image.open("headline_img_12.png")  # 512x64

for i, im_pair in enumerate(all_pairs[:200]):
    img1 = Image.open(im_pair[0])
    img2 = Image.open(im_pair[1])

    # Create a blank image (512x320)
    new_image = Image.new("RGB", (512, 512), (0, 0, 0))  # Black background

    # Paste images onto the new canvas
    # new_image.paste(headline, (0, 64))    # Headline at the top
    new_image.paste(img1, (0, 128))       # First image on the left
    new_image.paste(img2, (256, 128))     # Second image on the right

    # resize the image to 256x256
    # new_image = new_image.resize((256, 256), resample=Image.BICUBIC)
    # Save or display the final image
    new_image.save(F"combined_images/{i}_noh.png")


In [12]:
vlm_query = """
<|im_start|>system
You are an AI assistant that gets an image as input and answers a multiple choice question based on the image.
<|im_end|>

<|im_start|>user
<image>
question about the image:
Which of these two image best describes {character}?

A) Image on the left
B) Image on the right
Only output the letter of the choice (A or B).
<|im_end|>
<|im_start|>assistant
"""

all_questions = []
for i, im_pair in enumerate(all_pairs[:200]):
    name_1 = im_pair[0].name.split("_")[0]
    name_2 = im_pair[1].name.split("_")[0]

    all_questions.append(vlm_query.format(character=name_1))
    all_questions.append(vlm_query.format(character=name_2))

all_questions[0], all_pairs[0]

('\n<|im_start|>system\nYou are an AI assistant that gets an image as input and answers a multiple choice question based on the image.\n<|im_end|>\n\n<|im_start|>user\n<image>\nquestion about the image:\nWhich of these two image best describes The Asian woman?\n\nA) Image on the left\nB) Image on the right\nOnly output the letter of the choice (A or B).\n<|im_end|>\n<|im_start|>assistant\n',
 (PosixPath('../bias_images/bias_race/generated_images_race/race_12/The Asian woman_48.png'),
  PosixPath('../bias_images/bias_ses/generated_images_ses/ses_23/The sales clerk_13.png')))

In [46]:
model_id = "OpenGVLab/InternVL2_5-8B"
device  = "cuda:0"
batch_size = 32

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
tokenizer.padding_side = 'left'

model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().to(device)

IMG_START_TOKEN='<img>'
IMG_END_TOKEN='</img>'
IMG_CONTEXT_TOKEN='<IMG_CONTEXT>'
img_context_token_id = tokenizer.convert_tokens_to_ids(IMG_CONTEXT_TOKEN)
model.img_context_token_id = img_context_token_id

AB_token_ids = [290, 264, 309, 260] # A, a, B, b


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.85it/s]


In [13]:
pixel_values_list = []
image_queries = []
for i, question in enumerate(all_questions):
    image_tokens = IMG_START_TOKEN + IMG_CONTEXT_TOKEN * model.num_image_token + IMG_END_TOKEN
    question = question.replace('<image>', image_tokens, 1)
    image_queries.append(question)

    pixel_values_list.append(load_image(F"./combined_images/{i // 2}_noh.png", max_num=12).to(torch.bfloat16))


pixel_values_list[1].shape

torch.Size([1, 3, 448, 448])

In [ ]:
all_choices = []
for bi in tqdm(range(0, len(image_queries), batch_size)):
    query_batch = image_queries[bi:bi + batch_size]
    image_batch = torch.cat(pixel_values_list[bi:bi + batch_size], dim=0)

    model_inputs = tokenizer(query_batch, padding=True, return_tensors='pt')

    logits, probs = get_internvl_logits(model, image_batch, **model_inputs)
    probs = probs.cpu().detach()

    all_yes = probs[:, -1, AB_token_ids[:2]].sum(dim=-1).tolist()
    all_yes_nos = probs[:, -1, AB_token_ids].sum(dim=-1).tolist()

    for i in range(len(query_batch)):
        if all_yes[i] / all_yes_nos[i] > 0.5:
            all_choices.append("A")
        else:
            all_choices.append("B")

all_choices[:15]

100%|██████████| 13/13 [00:41<00:00,  3.16s/it]


['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A']

In [49]:
corrects = 0
for i, ch in enumerate(all_choices):
    if i % 2 == 0 and ch == "A":
        corrects += 1
    elif i % 2 == 1 and ch == "B":
        corrects += 1

corrects / len(all_choices)


0.9825

<h2>Test with generation</h2>

In [40]:
print(all_pairs[0])
print("\n\n" + image_queries[0][image_queries[0].find("question about the image:"):])
model_inputs = tokenizer(image_queries[0], return_tensors='pt').to(model.device)

generation_config = dict(max_new_tokens=4, do_sample=False)
generation_config['eos_token_id'] = tokenizer.eos_token_id

generation_output = model.generate(
            pixel_values=pixel_values_list[0].to(model.device),
            input_ids=model_inputs['input_ids'],
            attention_mask=model_inputs['attention_mask'],
            **generation_config
        )

response = tokenizer.batch_decode(generation_output, skip_special_tokens=True)[0]
print(generation_output)
print("response:", response)

logits, probs = get_logits(model, pixel_values_list[0], **model_inputs)
all_yes = probs[:, -1, AB_token_ids[:2]].sum(dim=-1).tolist()
all_yes_nos = probs[:, -1, AB_token_ids].sum(dim=-1).tolist()

# all_yes / all_yes_nos
print(all_yes, all_yes_nos)
for i in range(1):
    if all_yes[i] / all_yes_nos[i] > 0.5:
        print("A")
    else:
        print("B")


(PosixPath('../bias_images/bias_race/generated_images_race/race_12/The Asian woman_48.png'), PosixPath('../bias_images/bias_ses/generated_images_ses/ses_23/The sales clerk_13.png'))


question about the image:
Which of these two image best describes The Asian woman?

A) Image on the left
B) Image on the right
Only output the letter of the choice (A or B).
<|im_end|>
<|im_start|>assistant

tensor([[  290, 92542, 92542, 92542]], device='cuda:0')
response: A
[0.9959787726402283] [0.9966863989830017]
A
